In [1]:
import json
from pathlib import Path

import pandas as pd
import yaml
from loguru import logger

In [ ]:
df = pd.DataFrame(columns=["model"])

for day in Path("/Users/janpf/projects/Superkleber/outputs").iterdir():
    for run in day.iterdir():
        if day.name == "slurm_logs":
            continue
        logger.info(f"Processing {run}")
        if (run / "all_results.json").exists():
            with open(run / "all_results.json") as f:
                eval_results = json.load(f)
                logger.debug(eval_results)

            if (
                eval_results["train_samples"] == 10
                or eval_results["eval_samples"] == 10
                or eval_results["pred_samples"] == 10
            ):
                logger.info("Skipping debug run")
                continue

            with open(run / "hydra_conf/config.yaml") as f:
                cfg = yaml.load(f, Loader=yaml.FullLoader)
            columns = ["model", "task", "seed", "source"]
            if "predict_overall_f1" in eval_results:
                columns.append("predict_overall_f1")
            if "predict_overall_accuracy" in eval_results:
                columns.append("predict_overall_accuracy")
            if "eval_overall_f1" in eval_results:
                columns.append("eval_overall_f1")
            if "eval_accuracy" in eval_results:
                columns.append("eval_accuracy")

            if "predict_loss" in eval_results:
                columns.append("predict_loss")
            if "eval_loss" in eval_results:
                columns.append("eval_loss")

            print(columns)

            df = pd.concat(
                [
                    df,
                    pd.DataFrame(
                        data=[
                            [
                                cfg["model"]["model_name"],
                                cfg["task"]["task_name"],
                                cfg["seed"],
                                f"{day.name}/{run.name}",
                                *[eval_results[c] for c in columns if "_" in c],
                            ]
                        ],
                        columns=columns,
                    ),
                ]
            )
            del eval_results
            del cfg

df.sort_values(by=["task", "model", "seed"], inplace=True)

df